In [5]:
import re
import json

In [6]:
class ParsingError(Exception):
    """Exception personnalisée pour les erreurs de parsing."""
    def __init__(self, message, line_number=None, position=None):
        self.message = message
        self.line_number = line_number
        self.position = position
        super().__init__(message)

**Objectif** : 
Créer une exception spécifique pour les erreurs de parsing, qui inclut un message d'erreur et une position dans le code pour faciliter le débogage.

**Explications** :
Le constructeur prend un message et une position (facultative) pour indiquer où l'erreur s'est produite.
Cette exception sera utilisée plus tard pour capturer et signaler les erreurs lors de l'analyse du code.

In [7]:
# Classe principale du parser
class MTddVParser:
    def __init__(self, code):
        """Initialisation du parser avec le code à analyser."""
        self.code = code
        self.tokens = self.tokenize(code)  # Découper le code en tokens
        self.current_token_index = 0  # Index du token actuel
        self.errors = []  # Liste pour stocker les erreurs
        self.boucle_stack = []  # Pile pour gérer les blocs 'boucle' imbriqués

    def tokenize(self, code):
        """Tokeniser le code d'entrée en découpant sur les espaces blancs et les caractères spéciaux."""
        lines = code.splitlines()
        tokens = []
        for line_number, line in enumerate(lines, start=1):
            line_tokens = re.findall(r'\w+|[{}#()]|fin|%', line)
            for token in line_tokens:
                tokens.append({"token": token, "line_number": line_number})
        return tokens

    def current_token(self):
        """Retourner le token actuel ou None si aucun token n'est disponible."""
        if self.current_token_index < len(self.tokens):
            return self.tokens[self.current_token_index]["token"]
        return None

    def current_line_number(self):
        """Retourner le numéro de ligne du token actuel."""
        if self.current_token_index < len(self.tokens):
            return self.tokens[self.current_token_index]["line_number"]
        return None

    def advance(self):
        """Avancer à l'élément suivant dans la liste des tokens."""
        self.current_token_index += 1

    def parse(self):
        """Effectuer le parsing du code et construire la structure d'instructions."""
        structure = []  # Liste pour stocker les instructions validées
        while self.current_token():
            try:
                instruction = self.parse_instruction()  # Analyser une instruction
                if instruction:
                    structure.append(instruction)
            except ParsingError as e:
                # En cas d'erreur, ajouter l'erreur à la liste et avancer au token suivant
                self.errors.append({
                    "error": e.message,
                    "line_number": e.line_number,
                    "position": self.current_token_index
                })
                self.advance()

        # Vérifier les blocs 'boucle' non fermés
        if self.boucle_stack:
            self.errors.append({
                "error": "Unclosed 'boucle' block(s) detected.",
                "line_number": self.current_line_number(),
                "position": self.current_token_index
            })

        # Vérifier la présence du caractère '#' à la fin du programme
        if self.tokens and self.tokens[-1]["token"] != "#":
            self.errors.append({
                "error": "Missing '#' at the end of the program.",
                "line_number": self.tokens[-1]["line_number"],
                "position": len(self.tokens) - 1
            })

        return structure

    def parse_instruction(self):
        """Analyser une instruction individuelle."""
        token = self.current_token()  # Récupérer le token actuel

        if token in ["I", "P", "G", "D", "0", "1"]:
            line_number = self.current_line_number()
            self.advance()
            return {"type": "instruction", "value": token, "line_number": line_number}

        elif token == "%":  # Commentaire
            self.advance()
            return {"type": "comment"}

        elif token == "#":  # Indicateur de fin
            self.advance()
            if self.current_token():
                self.errors.append({
                    "error": "'#' must be the last instruction in the program.",
                    "line_number": self.current_line_number(),
                    "position": self.current_token_index
                })
            return {"type": "end_marker", "value": "#"}

        elif token == "si":  # Bloc conditionnel
            line_number = self.current_line_number()
            self.advance()
            condition = self.parse_condition()
            inner_structure = []
            while self.current_token() != "}":
                if self.current_token() is None:
                    raise ParsingError("Missing closing '}' for 'si' block", line_number)
                inner_structure.append(self.parse_instruction())
            self.advance()  # Consommer '}'
            return {"type": "si", "condition": condition, "content": inner_structure, "line_number": line_number}

        elif token == "boucle":  # Bloc boucle
            line_number = self.current_line_number()
            self.boucle_stack.append("boucle")
            self.advance()
            inner_structure = []
            while self.current_token() != "}":
                if self.current_token() is None:
                    raise ParsingError("Missing closing '}' for 'boucle' block", line_number)
                inner_structure.append(self.parse_instruction())
            self.boucle_stack.pop()
            self.advance()  # Consommer '}'
            return {"type": "boucle", "content": inner_structure, "line_number": line_number}

        elif token == "fin":
            line_number = self.current_line_number()
            if not self.boucle_stack:
                raise ParsingError("'fin' outside of any 'boucle' block", line_number)
            self.advance()
            return {"type": "fin", "line_number": line_number}

        else:
            line_number = self.current_line_number()
            raise ParsingError(f"Unknown instruction '{token}'", line_number)

    def parse_condition(self):
        """Analyser la condition dans une instruction 'si'."""
        if self.current_token() == "(":
            self.advance()
            condition = self.current_token()
            self.advance()
            if self.current_token() == ")":
                self.advance()
                return condition
            else:
                raise ParsingError("Expected ')' after condition in 'si'", self.current_line_number())
        raise ParsingError("Expected '(' after 'si'", self.current_line_number())

##### Constructeur __init__ :

**Objectif** : Initialiser l'instance du parser avec le code source à analyser.
**Détails** :
self.code : Le code source qui sera analysé.
self.tokens : Liste des tokens générée en découpant le code par la méthode tokenize.
self.current_token_index : Position actuelle dans la liste des tokens.
self.errors : Liste qui contiendra les erreurs rencontrées durant l'analyse.
self.boucle_stack : Pile utilisée pour suivre les blocs boucle imbriqués et s'assurer qu'ils sont correctement fermés.


#### Méthode tokenize :

**Objectif** : Récupérer le token actuel à analyser.
**Détails** :
Si l'index actuel est inférieur à la taille de la liste de tokens, retourne le token à cette position.
Sinon, retourne None pour indiquer la fin des tokens.


#### Méthode advance :

**Objectif** : Avancer à l'élément suivant dans la liste des tokens.
**Détails** :
Augmente simplement l'index de current_token_index pour passer au token suivant.


#### Méthode parse :

**Objectif** : Analyser tout le code et produire la structure d'instructions, tout en collectant les erreurs.
**Détails** :
structure : Liste pour stocker les instructions valides extraites du code.
Pour chaque token, parse_instruction() est appelée pour analyser les instructions.
Si une exception ParsingError est levée, l'erreur est ajoutée à la liste errors et l'analyse continue avec le token suivant.
Si des blocs boucle ne sont pas fermés, ou si le programme ne se termine pas par un #, des erreurs sont ajoutées.


#### Méthode parse_instruction :

**Objectif** : Analyser une instruction à partir du token actuel et renvoyer la structure de cette instruction.
**Détails** : Selon le token, plusieurs blocs de code différents sont activés pour analyser des instructions spécifiques comme :
Instructions de base (I, P, G, D, 0, 1) : Retourne une instruction de type "instruction".
Commentaires (%) : Ignore le commentaire et le marque comme tel.
Bloc conditionnel (si) : Analyse une instruction conditionnelle avec une condition entre parenthèses.
Bloc de boucle (boucle) : Gère les boucles imbriquées et s'assure qu'elles sont correctement fermées.
Fin de boucle (fin) : S'assure que fin apparaît bien dans un bloc boucle.

### Méthode parse_condition :

**Objectif** : Analyser la condition à l'intérieur d'un bloc si (si condition).
**Détails** :
Si la condition commence par ( et se termine par ), elle est extraite et renvoyée. Si la parenthèse fermante est manquante, une erreur est ajoutée.

In [8]:
# Charger et analyser le code depuis un fichier .ts
file_path = "err_programme_mtddv.ts"

with open(file_path, "r", encoding="utf-8") as file:
    code = file.read()

# Créer une instance du parser et analyser le code
parser = MTddVParser(code)
parsed_structure = parser.parse()

# Choisir le nom du fichier de sortie en fonction de la présence d'erreurs
if parser.errors:
    output_filename = "./results/parsed_structure_with_errors.json"
    for error in parser.errors:
        print(f"Erreur détectée : {error['error']} à la ligne {error['line_number']}")
else:
    output_filename = "./results/parsed_structure_valid.json"

# Créer le contenu JSON
output = {
    "parsed_structure": parsed_structure,
    "errors": parser.errors,
    "metadata": {
        "num_instructions": len(parsed_structure),
        "num_errors": len(parser.errors)
    }
}

# Sauvegarder le contenu dans le fichier approprié
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=4, ensure_ascii=False)

print(f"Parsed structure and detected errors have been saved to '{output_filename}'")

Erreur détectée : Unknown instruction 'buocle' à la ligne 8
Erreur détectée : 'fin' outside of any 'boucle' block à la ligne 10
Erreur détectée : Unknown instruction '}' à la ligne 10
Erreur détectée : Unknown instruction '}' à la ligne 11
Parsed structure and detected errors have been saved to './results/parsed_structure_with_errors.json'
